In [55]:
import os
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit
from pyspark.sql.functions import udf
from pyspark.sql.types import *
datasets_path = os.path.join('..','movie_recommendation_system', 'datasets')
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
tags_file = os.path.join(datasets_path, 'ml-latest', 'tags.csv')
links_file = os.path.join(datasets_path, 'ml-latest','links.csv')
movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')

In [56]:
pyspark

<module 'pyspark' from '/Users/tanay/opt/anaconda3/envs/movie/lib/python3.7/site-packages/pyspark/__init__.py'>

In [25]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
"""
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "50g") \
    .config("spark.submit.deployMode", "local") \
    .config("spark.executor.memory", "200g") \
    .config('spark.driver.maxResultSize', '200G') \
    .config('spark.executor.cores', '5')\
    .config('spark.memory.fraction ', '0')\
    .config('spark.memory.offHeap.enabled', 'true')\
    .config('spark.memory.offHeap.size', '2g')\
    .appName('movie-recomendation') \
    .getOrCreate()
"""

'\nspark = SparkSession.builder     .master(\'local[*]\')     .config("spark.driver.memory", "50g")     .config("spark.submit.deployMode", "local")     .config("spark.executor.memory", "200g")     .config(\'spark.driver.maxResultSize\', \'200G\')     .config(\'spark.executor.cores\', \'5\')    .config(\'spark.memory.fraction \', \'0\')    .config(\'spark.memory.offHeap.enabled\', \'true\')    .config(\'spark.memory.offHeap.size\', \'2g\')    .appName(\'movie-recomendation\')     .getOrCreate()\n'

In [26]:
sc.getConf().getAll()

[('spark.app.name', 'Spark_Processor'),
 ('spark.jars',
  'spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar'),
 ('spark.app.id', 'local-1617679275779'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', '10.4.0.81'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.redis.port', '6379'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '64039')]

In [27]:
# conf1 = pyspark.SparkConf().setAll([('spark.executor.memory', '50g'), ('spark.executor.cores', '5'), ('spark.cores.max', '5'), ('spark.driver.memory','50g'), ('spark.memory.fraction', '0.01')])
# sc.stop()
# conf1 = pyspark.SparkConf().set("spark.jars", "/spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar")
# sc = pyspark.SparkContext(conf=conf1)
# spark = SparkSession.builder.config(conf=conf1).getOrCreate()
# sc.getConf().getAll()

spark = SparkSession.builder.appName("Spark_Processor").config("spark.redis.port", "6379").config("spark.jars", "spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar").getOrCreate()
conf = pyspark.SparkConf()
sc = spark.sparkContext
sc.getConf().getAll()

[('spark.app.name', 'Spark_Processor'),
 ('spark.jars',
  'spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar'),
 ('spark.app.id', 'local-1617679275779'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', '10.4.0.81'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.redis.port', '6379'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '64039')]

In [87]:
#Schema design which are as same as csv
ratingschema = StructType()\
    .add("userId", IntegerType(), True)\
    .add("movieId", IntegerType(), True)\
    .add("rating", DoubleType(), True)\
    .add("timeStamp", IntegerType(), True)
tagsschema = StructType()\
    .add("userId", IntegerType(), True)\
    .add("movieId", IntegerType(), True)\
    .add("tag", StringType(), True)\
    .add("timeStamp", IntegerType(), True)
linkschema = StructType()\
    .add("movieId", IntegerType(), True)\
    .add("IMDBID", IntegerType(), True)\
    .add("TMDBID", IntegerType(), True)

In [88]:
ratingdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(ratingschema)\
    .load(complete_ratings_file)
tagdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(tagsschema)\
    .load(tags_file)
linkdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(linkschema)\
    .load(links_file)

In [89]:
sc.getConf().getAll()

[('spark.app.name', 'Spark_Processor'),
 ('spark.jars',
  'spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar'),
 ('spark.app.id', 'local-1617679275779'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', '10.4.0.81'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.redis.port', '6379'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '64039')]

In [90]:
ratingdf.show(), tagdf, linkdf

+------+-------+------+----------+
|userId|movieId|rating| timeStamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



(None,
 DataFrame[userId: int, movieId: int, tag: string, timeStamp: int],
 DataFrame[movieId: int, IMDBID: int, TMDBID: int])

In [91]:
movie = spark.read.option("inferSchema", "true")\
    .option("header", "true").csv(movies_file)
movie

DataFrame[movieId: int, title: string, genres: string]

In [92]:
moviedf = movie.drop("genres")
genredf = movie.drop("title")
userdf = ratingdf.drop("movieId", "rating", "timeStamp")
userdf = userdf.select('userId').distinct()

In [93]:
moviedf, genredf, userdf, ratingdf

(DataFrame[movieId: int, title: string],
 DataFrame[movieId: int, genres: string],
 DataFrame[userId: int],
 DataFrame[userId: int, movieId: int, rating: double, timeStamp: int])

In [94]:
def ratingKey(user):
    out = "UserID: {},MovieID:".format(user)
    return out
udfRatingKey = udf(ratingKey, StringType())

In [95]:
moviedf = moviedf.select("title", "movieId")
print(moviedf.count())
moviedf, _ = moviedf.randomSplit([0.1, 0.8])
print(moviedf.count())

ratingdf = ratingdf.drop("timestamp")
print(ratingdf.count())
ratingdf = ratingdf.withColumn("key", (concat(lit("User ID: "),col("userId"),lit(", Movie ID: "),col("movieId"))))
ratingdf = ratingdf.select("key", "userId", "movieId", "rating")
ratingdf = ratingdf.join(moviedf, on=['movieId'], how='left_semi')
print(ratingdf.count())

likedMoviedf = ratingdf.filter(ratingdf.rating > 4.0)
# likedMoviedf.show()

62423
6926
25000095
2569535


In [51]:
moviedf.write.format("org.apache.spark.sql.redis").option("table", "movie").option("key.column", "title").save()

In [54]:
ratingdf.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()

Py4JJavaError: An error occurred while calling o936.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 42.0 failed 1 times, most recent failure: Lost task 1.0 in stage 42.0 (TID 65, localhost, executor driver): redis.clients.jedis.exceptions.JedisConnectionException: java.net.SocketTimeoutException: Read timed out
	at redis.clients.jedis.util.RedisInputStream.ensureFill(RedisInputStream.java:205)
	at redis.clients.jedis.util.RedisInputStream.readByte(RedisInputStream.java:43)
	at redis.clients.jedis.Protocol.process(Protocol.java:155)
	at redis.clients.jedis.Protocol.read(Protocol.java:220)
	at redis.clients.jedis.Connection.readProtocolWithCheckingBroken(Connection.java:318)
	at redis.clients.jedis.Connection.getMany(Connection.java:330)
	at redis.clients.jedis.Pipeline.sync(Pipeline.java:98)
	at com.redislabs.provider.redis.util.PipelineUtils$$anonfun$foreachWithPipeline$1.apply(PipelineUtils.scala:71)
	at com.redislabs.provider.redis.util.Pipe

In [ ]:
likedMoviedf.write.format("org.apache.spark.sql.redis").option("table", "likedMovie").option("key.column", "key").save()

In [53]:
read_moviedf = spark.read.format("org.apache.spark.sql.redis").option("table", "movie").option("key.column", "title").load()
read_moviedf.sort("title").show()

read_ratingdf = spark.read.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").load()
read_moviedf.sort("key").show()

read_likedMoviedf = spark.read.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").load()
read_likedMoviedf.sort("key").show()


Py4JJavaError: An error occurred while calling o910.load.
: java.lang.IllegalStateException: Unable to read dataframe schema by key '_spark:movie:schema'. If dataframe was not persisted by Spark, provide a schema explicitly with .schema() or use 'infer.schema' option. 
	at org.apache.spark.sql.redis.RedisSourceRelation.org$apache$spark$sql$redis$RedisSourceRelation$$loadSchema(RedisSourceRelation.scala:265)
	at org.apache.spark.sql.redis.RedisSourceRelation$$anonfun$schema$1.apply(RedisSourceRelation.scala:112)
	at org.apache.spark.sql.redis.RedisSourceRelation$$anonfun$schema$1.apply(RedisSourceRelation.scala:112)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.sql.redis.RedisSourceRelation.schema(RedisSourceRelation.scala:111)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:403)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:223)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scal

we need to connect Redis here

# ALS: collaborative filtering for explicit data

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [ ]:
ratingforals = ratingdf.drop("timeStamp")
ratingforals.show()
#spark.driver.memoryOverhead
#spark.executor.memoryOverhead

In [ ]:
(training, test) = ratingforals.randomSplit([0.5, 0.5])
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# c
als = ALS(maxIter=3, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", 
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

error message: 
1. when executor = 50g, 200g: Job aborted due to stage failure: Task 1 in stage 7.0 failed 1 times, most recent failure: Lost task 1.0 in stage 7.0 (TID 22, 192.168.1.73, executor driver): java.lang.OutOfMemoryError: Java heap space
2. additional maxreusltsize = 20G: Py4JJavaError: An error occurred while calling o145.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 7.0 failed 1 times, most recent failure: Lost task 6.0 in stage 7.0 (TID 27, 192.16An error occurred while calling o381.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 21, 192.168.1.73, executor driver): java.lang.OutOfMemoryError: Java heap space8.1.73, executor driver): java.lang.OutOfMemoryError: Java heap space\
maybe this:
https://knowledge.informatica.com/s/article/567632?language=en_US\
An error occurred while calling o381.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 21, 192.168.1.73, executor driver): java.lang.OutOfMemoryError: Java heap space